313549 ~ Mateusz Zacharecki \
335719 ~ Patrycja Kielan

---

# Solution for variable selection from model
___

# Importing libraries and data exploration

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import sklearn.model_selection as skm
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectFromModel
import xgboost
from sklearn.ensemble import VotingRegressor

C:\Users\XXX\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\XXX\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
X_train = pd.read_csv("Xtrain.csv", index_col=0)
X_test = pd.read_csv("Xtest.csv", index_col=0)
y_train = pd.read_csv("ytrain.csv", index_col=0)

# SVM

In [3]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

## Fitting the whole training set

In [4]:
pipeline_SVM = Pipeline([
    ('pipeline', pipeline),
    ('svm', SVR(C = 10, gamma = 0.1, kernel = 'rbf'))
])

In [5]:
params = {}

grid_pipeline_SVM = GridSearchCV(pipeline_SVM,
                             cv = skm.KFold(3, random_state = 313549, shuffle=True),
                             param_grid = params,
                             scoring = 'neg_mean_squared_error',
                             return_train_score = True)

In [6]:
grid_pipeline_SVM.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('svm', SVR(C=10, gamma=0.01))]),
             param_grid={}, return_train_score=True,
             scoring='neg_mean_squared_error')

In [7]:
results_SVM = pd.DataFrame({
                     'mean_test_reg_mean_squared_error': grid_pipeline_SVM.cv_results_["mean_test_score"],
                     'mean_train_reg_mean_squared_error': grid_pipeline_SVM.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_SVM.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_SVM.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_SVM.cv_results_["rank_test_score"]})
results_SVM.sort_values(by = 'rank_test')

,mean_test_reg_mean_squared_error,mean_train_reg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-91.047322,-90.352973,1.378589,0.448389,1


In [8]:
y_pred = pd.DataFrame(grid_pipeline_SVM.predict(X_test))

In [9]:
pd.DataFrame(y_pred).to_csv("SVM.txt", sep='\t', index=False, header=False)

# Neural network

In [9]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

## Fitting the whole training set

In [10]:
pipeline_NN = Pipeline([
    ('pipeline', pipeline),
    ('nn', MLPRegressor(max_iter = 10000, random_state = 313549))
])

In [11]:
params = {
    "nn__hidden_layer_sizes": [(4), (16), (4, 4)],
    "nn__solver": ['sgd', 'adam']
}

In [12]:
grid_pipeline_NN = GridSearchCV(pipeline_NN,
                      cv = skm.KFold(5, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [13]:
 grid_pipeline_NN.fit(X_train, y_train.values.ravel())

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", 

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 96, in _check_reg_targets
    y_pred = check_a

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\utils\extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", 

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "C:\Users\XXX\anaconda3\lib\site-packages\sklearn\metrics\_regression.py", line 96, in _check_reg_targets
    y_pred = check_a

C:\Users\XXX\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [          nan -100.65676017           nan  -94.88326144 -120.20342969
 -104.0673914 ]
  warnings.warn(
C:\Users\XXX\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the train scores are non-finite: [          nan  -97.02434287           nan  -93.65345988 -120.20417868
 -102.57804474]
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('nn',
                                        MLPRegressor(max_iter=10000,
                                                     random_state=313549))]),
             param_grid={'nn__hidden_layer_sizes': [4, 16, (4, 4)],
                         'nn__solver': ['sgd', 'adam']},
             return_train_score=True, scoring='neg_mean_squared_error')

In [14]:
results_NN = pd.DataFrame({'nn__hidden_layer_sizes': grid_pipeline_NN.cv_results_["param_nn__hidden_layer_sizes"],
                           'nn__solver': grid_pipeline_NN.cv_results_["param_nn__solver"],
                     'mean_test_reg_mean_squared_error': grid_pipeline_NN.cv_results_["mean_test_score"],
                     'mean_train_reg_mean_squared_error': grid_pipeline_NN.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_NN.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_NN.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_NN.cv_results_["rank_test_score"]})
results_NN.sort_values(by = 'rank_test')

,nn__hidden_layer_sizes,nn__solver,mean_test_reg_mean_squared_error,mean_train_reg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
3,16,adam,-94.883261,-93.653460,1.007904,1.397966,1
1,4,adam,-100.656760,-97.024343,3.831777,0.434354,2
5,"(4, 4)",adam,-104.067391,-102.578045,9.205217,8.627271,3
4,"(4, 4)",sgd,-120.203430,-120.204179,1.356512,0.340512,4
0,4,sgd,NaN,NaN,NaN,NaN,5
2,16,sgd,NaN,NaN,NaN,NaN,6


In [15]:
y_pred = pd.DataFrame(grid_pipeline_NN.predict(X_test))

In [16]:
pd.DataFrame(y_pred).to_csv("neural_network.txt", sep='\t', index=False, header=False)

# XGBoost

In [3]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

## Fitting the whole training set

In [4]:
pipeline_xgb = Pipeline([
    ('pipeline', pipeline),
    ('xgb', xgboost.XGBRegressor(objective='reg:squarederror', sampling_method = "hist", n_estimators = 400, random_state = 313549))
])

In [5]:
params = {
    "xgb__eta": [0.3, 0.1, 0.2],
    "xgb__gamma": [0, 0.1, 1],
    "xgb__max_depth": [10, 20]
}

In [6]:
grid_pipeline_xgb = GridSearchCV(pipeline_xgb,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [7]:
grid_pipeline_xgb.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('xgb',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                  

In [8]:
results_xgb = pd.DataFrame({'xgb__eta': grid_pipeline_xgb.cv_results_["param_xgb__eta"],
                            'xgb__gamma': grid_pipeline_xgb.cv_results_["param_xgb__gamma"],
                            'xgb__max_depth': grid_pipeline_xgb.cv_results_["param_xgb__max_depth"],
                     'mean_test_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_xgb.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_xgb.cv_results_["rank_test_score"]})
results_xgb.sort_values(by = 'rank_test')

,xgb__eta,xgb__gamma,xgb__max_depth,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
10,0.1,1,10,-86.033086,-3.317566e+01,1.256510,2.073159e+00,1
6,0.1,0,10,-86.220520,-3.363248e+01,0.900189,4.227421e-01,2
8,0.1,0.1,10,-86.460972,-3.394905e+01,1.398941,3.107234e+00,3
9,0.1,0.1,20,-89.071144,-1.060868e-01,1.361065,4.681314e-02,4
11,0.1,1,20,-89.261594,-2.243307e-01,0.977953,3.955445e-02,5
7,0.1,0,20,-89.347817,-1.186002e-01,1.490601,9.007067e-02,6
12,0.2,0,10,-90.510123,-1.697458e+01,0.999454,2.051259e+00,7
16,0.2,1,10,-90.613563,-1.702242e+01,0.746363,8.240583e-01,8
14,0.2,0.1,10,-90.721735,-1.727156e+01,0.895387,1.433064e+00,9
13,0.2,0,20,-94.133068,-1.215807e-04,1.160995,9.815152e-05,10


In [9]:
y_pred = pd.DataFrame(grid_pipeline_xgb.predict(X_test))

In [10]:
pd.DataFrame(y_pred).to_csv("xgboost.txt", sep='\t', index=False, header=False)

# XGBoost for eta = 0.03

In [3]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

## Fitting the whole training set

In [4]:
pipeline_xgb003 = Pipeline([
    ('pipeline', pipeline),
    ('xgb', xgboost.XGBRegressor(objective='reg:squarederror', sampling_method = "hist", n_estimators = 400, random_state = 313549))
])

In [5]:
params = {
    "xgb__eta": [0.03],
    "xgb__gamma": [0, 0.1, 1],
    "xgb__max_depth": [10, 20]
}

In [6]:
grid_pipeline_xgb003 = GridSearchCV(pipeline_xgb003,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [7]:
grid_pipeline_xgb003.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('xgb',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                  

In [8]:
results_xgb003 = pd.DataFrame({'xgb__eta': grid_pipeline_xgb003.cv_results_["param_xgb__eta"],
                            'xgb__gamma': grid_pipeline_xgb003.cv_results_["param_xgb__gamma"],
                            'xgb__max_depth': grid_pipeline_xgb003.cv_results_["param_xgb__max_depth"],
                     'mean_test_neg_mean_squared_error': grid_pipeline_xgb003.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_xgb003.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_xgb003.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_xgb003.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_xgb003.cv_results_["rank_test_score"]})
results_xgb003.sort_values(by = 'rank_test')

,xgb__eta,xgb__gamma,xgb__max_depth,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
4,0.03,1,10,-84.220419,-58.366194,1.305361,1.705480,1
2,0.03,0.1,10,-84.445759,-58.643519,1.203526,1.639037,2
0,0.03,0,10,-84.722289,-59.738044,0.986824,0.884374,3
5,0.03,1,20,-85.636130,-7.227130,0.976795,1.133446,4
1,0.03,0,20,-85.827005,-8.386968,1.333895,2.382029,5
3,0.03,0.1,20,-86.023198,-9.176881,1.158006,1.793172,6


In [9]:
y_pred = pd.DataFrame(grid_pipeline_xgb003.predict(X_test))

In [10]:
pd.DataFrame(y_pred).to_csv("xgboost003.txt", sep='\t', index=False, header=False)

# Linear regression

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

## Fitting the whole training set

In [10]:
pipeline_lm = Pipeline([
    ('pipeline', pipeline),
    ('lm', LinearRegression())
])

In [11]:
params = {}

In [12]:
grid_pipeline_lm = GridSearchCV(pipeline_lm,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [13]:
grid_pipeline_lm.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('lm', LinearRegression())]),
             param_grid={}, return_train_score=True,
             scoring='neg_mean_squared_error')

In [14]:
results_lm = pd.DataFrame({
                     'mean_test_neg_mean_squared_error': grid_pipeline_lm.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_lm.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_lm.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_lm.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_lm.cv_results_["rank_test_score"]})
results_lm.sort_values(by = 'rank_test')

,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-96.14743,-96.129248,1.235674,0.177028,1


In [15]:
y_pred = pd.DataFrame(grid_pipeline_lm.predict(X_test))

In [16]:
pd.DataFrame(y_pred).to_csv("linear_regression.txt", sep='\t', index=False, header=False)

# Voting

In [11]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

## Fitting the whole data set

In [14]:
reg1 = SVR(C = 10, gamma = 0.1, kernel = 'rbf')
reg2 = MLPRegressor(hidden_layer_sizes = 16, solver = 'adam', max_iter = 10000, random_state = 313549)
reg3 = xgboost.XGBRegressor(eta = 0.1, gamma = 1, max_depth = 10, objective='reg:squarederror', sampling_method = "hist", n_estimators = 400, random_state = 313549)

pipeline_voting = Pipeline([
    ('pipeline', pipeline),
    ('eclf', VotingRegressor(estimators=[('svm', reg1),
                             ('nn', reg2),
                             ('xgb', reg3)]))
])

In [16]:
params = {}

grid_pipeline_voting = GridSearchCV(pipeline_voting,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [17]:
grid_pipeline_voting.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('eclf',
                                        VotingRegressor(estimators=[('svm',
                                                                     SVR(C=10,
                                                                         gamma=0.1)),
                                                                    ('nn',
                                                                     MLPRegressor(hidden_layer_size

In [18]:
results_voting = pd.DataFrame({
                     'mean_test_neg_mean_squared_error': grid_pipeline_voting.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_voting.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_voting.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_voting.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_voting.cv_results_["rank_test_score"]})
results_voting.sort_values(by = 'rank_test')

,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-84.741533,-59.502059,1.471369,1.942283,1


In [19]:
y_pred = pd.DataFrame(grid_pipeline_voting.predict(X_test))

In [20]:
pd.DataFrame(y_pred).to_csv("voting.txt", sep='\t', index=False, header=False)

# Voting for other XGBoost parameters

In [11]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(DecisionTreeRegressor(), max_features = 20))
])

In [15]:
reg1 = SVR(C = 10, gamma = 0.1, kernel = 'rbf')
reg2 = MLPRegressor(hidden_layer_sizes = 16, solver = 'adam', max_iter = 10000, random_state = 313549)
reg3 = xgboost.XGBRegressor(eta = 0.03, gamma = 1, max_depth = 10, objective='reg:squarederror', sampling_method = "hist", n_estimators = 400, random_state = 313549)

pipeline_voting003 = Pipeline([
    ('pipeline', pipeline),
    ('eclf', VotingRegressor(estimators=[('svm', reg1),
                             ('nn', reg2),
                             ('xgb', reg3)]))
])

In [17]:
params = {}

grid_pipeline_voting003 = GridSearchCV(pipeline_voting003,
                      cv = skm.KFold(3, random_state = 313549, shuffle=True),
                      param_grid = params,
                      scoring = 'neg_mean_squared_error',
                      return_train_score = True)

In [18]:
grid_pipeline_voting003.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=KFold(n_splits=3, random_state=313549, shuffle=True),
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler()),
                                                        ('selector',
                                                         SelectFromModel(estimator=DecisionTreeRegressor(),
                                                                         max_features=20))])),
                                       ('eclf',
                                        VotingRegressor(estimators=[('svm',
                                                                     SVR(C=10,
                                                                         gamma=0.1)),
                                                                    ('nn',
                                                                     MLPRegressor(hidden_layer_size

In [19]:
results_voting003 = pd.DataFrame({
                     'mean_test_neg_mean_squared_error': grid_pipeline_voting003.cv_results_["mean_test_score"],
                     'mean_train_neg_mean_squared_error': grid_pipeline_voting003.cv_results_["mean_train_score"],
                     'std_test_neg_mean_squared_error': grid_pipeline_voting003.cv_results_["std_test_score"],
                     'std_train_neg_mean_squared_error': grid_pipeline_voting003.cv_results_["std_train_score"],
                     'rank_test': grid_pipeline_voting003.cv_results_["rank_test_score"]})
results_voting003.sort_values(by = 'rank_test')

,mean_test_neg_mean_squared_error,mean_train_neg_mean_squared_error,std_test_neg_mean_squared_error,std_train_neg_mean_squared_error,rank_test
0,-84.694381,-70.44955,0.76407,1.506481,1


In [20]:
y_pred = pd.DataFrame(grid_pipeline_voting003.predict(X_test))

In [21]:
pd.DataFrame(y_pred).to_csv("voting003.txt", sep='\t', index=False, header=False)